In [47]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
import numpy as np
import pandas as pd
import PreClf, MultiPreClf
from sklearn.datasets import fetch_openml
from sklearn.datasets import make_moons

import matplotlib.pyplot as plt 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

# MLP from sklearn
from sklearn.neural_network import MLPClassifier


# MNIST 
X_, y_ = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False, parser="pandas")
# Preprocessing normalisation 
X_ = X_ / 255

y_ = y_.astype(int)

# On-hot encoding
encoder = OneHotEncoder(sparse=False)
y_one_hot = encoder.fit_transform(y_.reshape(-1, 1))


X_train, X_test, y_train, y_test = train_test_split(X_, y_one_hot, test_size=0.2, random_state=42)


sk_mlp = MLPClassifier(hidden_layer_sizes=(100, 100), activation='relu', solver='adam', alpha=0.0001, batch_size='auto',
                          learning_rate='constant', learning_rate_init=0.001, power_t=0.5, max_iter=200, shuffle=True,
                            random_state=None, tol=0.0001, verbose=False, warm_start=False, momentum=0.9,
                                nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.9,
                                    beta_2=0.999, epsilon=1e-08, n_iter_no_change=10, max_fun=15000)


m:\ML\ML_regs\.venv\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [66]:
model = MultiPreClf.MultilayerPerceptron(alpha=0.01, 
                                         n_iterations=2, 
                                         input_layer_size=784, 
                                         hidden_layers_size=[32, 16], 
                                         n_outputs=10, 
                                         regularization="l2", 
                                         lambda_reg=0.1)

model_two = MultiPreClf.MLP(alpha=0.01, n_iterations=5, 
                            input_layer_size=784, hidden_layers_size=[32, 16], 
                            n_outputs=10,
                            regularization="l2", 
                            lambda_reg=0.1)

In [63]:
a = model_two.forward_propagation(X_train[:5, :])

In [64]:
a[-1]

array([[0.1196863 , 0.1474748 , 0.10284153, 0.0989867 , 0.04712092,
        0.10836891, 0.06494019, 0.15305338, 0.06896177, 0.0885655 ],
       [0.08492067, 0.20089706, 0.12936984, 0.11301478, 0.01573648,
        0.09077563, 0.02110054, 0.20330906, 0.08255924, 0.05831669],
       [0.10102529, 0.1726681 , 0.11781825, 0.08946777, 0.03457797,
        0.09614729, 0.05655688, 0.17538551, 0.06679194, 0.08956101],
       [0.13054328, 0.12606271, 0.078882  , 0.11691978, 0.05149726,
        0.06184108, 0.04572222, 0.25778927, 0.04243273, 0.08830967],
       [0.06540318, 0.20579222, 0.0934958 , 0.0533974 , 0.0130552 ,
        0.07630971, 0.02309069, 0.37991942, 0.04224762, 0.04728875]])

In [65]:
model_two.backpropagation(X_train[:5, :], y_train[:5, :], a)

In [62]:
e.shape

(5, 10)

In [ ]:
training_loss, validation_loss = model.train(X_train, y_train, X_test, y_test)


In [34]:
# Step 3: Evaluate the Model
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Step 4: Calculate Accuracy
accuracy = model.accuracy(y_test_classes, y_pred_classes)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 11.43%


In [37]:
y_pred

array([[0.0984442 , 0.11360162, 0.09985351, ..., 0.10101594, 0.09880851,
        0.09915438],
       [0.0984442 , 0.11360162, 0.09985351, ..., 0.10101594, 0.09880851,
        0.09915438],
       [0.0984442 , 0.11360162, 0.09985351, ..., 0.10101594, 0.09880851,
        0.09915438],
       ...,
       [0.0984442 , 0.11360162, 0.09985351, ..., 0.10101594, 0.09880851,
        0.09915438],
       [0.0984442 , 0.11360162, 0.09985351, ..., 0.10101594, 0.09880851,
        0.09915438],
       [0.0984442 , 0.11360162, 0.09985351, ..., 0.10101594, 0.09880851,
        0.09915438]])

In [35]:
y_pred_classes

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [36]:
y_test_classes

array([8, 4, 8, ..., 2, 7, 1], dtype=int64)

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt


In [6]:
train = pd.read_csv("C:/Users/denis/Downloads/digit-recognizer/train.csv")
test = pd.read_csv("C:/Users/denis/Downloads/digit-recognizer/test.csv")

In [9]:
data = np.array(train)

m , n = data.shape

In [10]:
np.random.shuffle(data)

In [11]:
data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]

In [15]:
def  init_params():

    W_one = np.random.rand(10, 784) - 0.5
    b_one = np.random.rand(10, 1) - 0.5

    W_two = np.random.rand(10, 10) - 0.5
    b_two = np.random.rand(10, 1) - 0.5

    return W_one, b_one, W_two, b_two

def relu(Z):

    return np.maximum(0, Z)

def softmax(Z):

    A = np.exp(Z) / sum(np.exp(Z))

    return A

def forward_prop(W_one, b_one, W_two, b_two, X):

    Z_one = W_one.dot(X) + b_one
    A_one = relu(Z_one)

    Z_two = W_two.dot(A_one) + b_two
    A_two = softmax(Z_two)

    return Z_one, A_one, Z_two, A_two